In [1]:
import pandas as pd
import numpy as np

In [2]:
salary_data = pd.read_excel("salaire.xls", skiprows=5)
immigration_data = pd.read_excel("immigration.xls", skiprows=10)
population_data = pd.read_excel("population.xls", skiprows=5)

### First, as usual, we convert unicode to string.

In [3]:
import unicodedata

def unicode_to_string(word):
    if pd.isnull(word):
        return 
    else:
        return unicodedata.normalize('NFKD', word).encode('ascii','ignore')
    
def data_to_string(data):
    new_data = data.copy()
    
    col_title_unicode = new_data.columns.values.tolist()
    col_title = map(unicode_to_string,col_title_unicode)
    new_data.columns = col_title
    
    for col in new_data.columns:
        not_nan_index = [not ind for ind in new_data[col].isnull()]
        not_nan_value = new_data[col][not_nan_index]
        if type(not_nan_value.iloc[0]) == unicode: #check the first not-NaN value
            new_data[col] = map(unicode_to_string,new_data[col])
            
    return new_data

In [4]:
salary_data = data_to_string(salary_data)
immigration_data = data_to_string(immigration_data)
population_data = data_to_string(population_data)

### Now we will rename the column headers so that we have at least 1 common column for all the data frames. 

In [5]:
def rename_column(data): 
    new_data = data.copy()
    col_title = new_data.columns.tolist()
    for x in xrange(len(col_title)):
        if col_title[x] == 'CODGEO':
            col_title[x] = 'Code Insee' # this will be the pivot column for merging
        if col_title[x] == 'LIBGEO':
            col_title[x] = 'Libelle de la commune'
    new_data.columns = col_title
    return new_data

In [6]:
salary_data = rename_column(salary_data)
immigration_data = rename_column(immigration_data)
population_data = rename_column(population_data)

### We still have problem with departemental code of Corse because they are not in numerical form (2A... or 2B...).

In [7]:
def to_digit(string):
    new_string = list(string)[:]
    for x in xrange(len(new_string)): 
        if not new_string[x].isdigit():
            new_string[x] = '0'
    return int(''.join(new_string))

In [8]:
def replace_insee_code(data): #replace 2AXXX or 2BXXX by 20XXX 
        new_data = data.copy()
        insee_list = new_data['Code Insee'].tolist() 
        new_insee_list = [int(code) if code.isdigit() else to_digit(code) for code in insee_list]
        new_data['Code Insee'] = new_insee_list
        return new_data

In [9]:
salary_data = replace_insee_code(salary_data)
immigration_data = replace_insee_code(immigration_data)
population_data = replace_insee_code(population_data)

### There is still another problem: all the numbers in our excel sheet are read in float format. We need to convert them to int for the immigration and population data.

In [10]:
def float_to_int(data):
    new_data = data.copy()    
    for col in new_data.columns:
        not_nan_index = [not ind for ind in new_data[col].isnull()]
        not_nan_value = new_data[col][not_nan_index]
        if type(not_nan_value.iloc[0]) == np.float64: #check the first not-NaN value
            new_data[col] = new_data[col].round()
    return new_data

In [11]:
immigration_data = float_to_int(immigration_data)
population_data = float_to_int(population_data)

### In immigration data, we will add a column which shows us the percentage of adult immigrants  in each town.

In [12]:
def add_total_immigrant_variable(data): 
    new_data = data.copy()
    variable_list = []
    sum_list = []
    for variable in data.columns.tolist(): 
        if 'IMMI' in variable and 'AGE400' not in variable: 
            sum_list.append(variable)
            if 'IMMI1' in variable:
                variable_list.append(variable)
    new_data['% immigrant'] = (new_data[variable_list].sum(axis=1)/ new_data[sum_list].sum(axis=1)) * 100
    col = new_data.columns.tolist()
    col = col[:2] + col[-1:] + col[2:-1]
    new_data = new_data[col]
    return new_data

In [13]:
immigration_data = add_total_immigrant_variable(immigration_data)

In [14]:
immigration_data.head()

,Code Insee,Libelle de la commune,% immigrant,AGE400_IMMI1_SEXE1,AGE400_IMMI1_SEXE2,AGE400_IMMI2_SEXE1,AGE400_IMMI2_SEXE2,AGE415_IMMI1_SEXE1,AGE415_IMMI1_SEXE2,AGE415_IMMI2_SEXE1,AGE415_IMMI2_SEXE2,AGE425_IMMI1_SEXE1,AGE425_IMMI1_SEXE2,AGE425_IMMI2_SEXE1,AGE425_IMMI2_SEXE2,AGE455_IMMI1_SEXE1,AGE455_IMMI1_SEXE2,AGE455_IMMI2_SEXE1,AGE455_IMMI2_SEXE2
0,1001,L'Abergement-Clemenciat,4.173623,1,3,92,83,1,0,34,32,7,9,154,137,5,3,101,116
1,1002,L'Abergement-de-Varey,2.105263,0,1,25,18,1,0,6,11,2,1,52,47,0,0,38,32
2,1004,Amberieu-en-Bugey,12.048946,57,53,1386,1542,73,61,802,872,442,403,2447,2386,194,176,1454,1886
3,1005,Amberieux-en-Dombes,4.528012,0,0,176,163,0,2,108,101,18,19,334,324,11,9,182,195
4,1006,Ambleon,8.000000,2,1,5,4,0,0,8,3,2,3,23,19,0,3,18,21


### In population data, we will transform the social status columns from quantity to percentage too.

In [15]:
def social_status_percentage(data): 
    new_data = data.copy()
    status_list = ['C12_POP15P_CS1', 'C12_POP15P_CS2','C12_POP15P_CS3','C12_POP15P_CS4','C12_POP15P_CS5',
                   'C12_POP15P_CS6','C12_POP15P_CS7','C12_POP15P_CS8']
    adult_population = new_data['C12_POP15P']
    for col in status_list:
        new_data['% '+ col] = (new_data[col] / adult_population)*100
    return new_data

In [16]:
population_data = social_status_percentage(population_data)

In [17]:
population_data

,Code Insee,REG,DEP,Libelle de la commune,P12_POP,P12_POP0014,P12_POP1529,P12_POP3044,P12_POP4559,P12_POP6074,...,C12_POP55P_CS7,C12_POP55P_CS8,% C12_POP15P_CS1,% C12_POP15P_CS2,% C12_POP15P_CS3,% C12_POP15P_CS4,% C12_POP15P_CS5,% C12_POP15P_CS6,% C12_POP15P_CS7,% C12_POP15P_CS8
0,1001,82,01,L'Abergement-Clemenciat,777,178,97,165,151,128,...,163,8,2.588997,0.647249,12.944984,14.239482,18.770227,11.650485,27.022654,12.297735
1,1002,82,01,L'Abergement-de-Varey,235,44,35,60,38,36,...,64,0,0.000000,2.116402,10.582011,27.513228,4.232804,16.931217,33.862434,4.232804
2,1004,82,01,Amberieu-en-Bugey,14233,3038,2904,2722,2655,1737,...,2816,251,0.000000,2.818540,6.451324,15.595920,17.143880,17.197566,25.590551,15.211167
3,1005,82,01,Amberieux-en-Dombes,1642,338,301,348,359,203,...,321,33,0.000000,5.611068,8.147579,17.524981,18.139892,14.681015,25.288240,10.607225
4,1006,82,01,Ambleon,110,12,18,24,27,20,...,35,0,0.000000,0.000000,4.651163,18.604651,18.604651,9.302326,40.697674,9.302326
5,1007,82,01,Ambronay,2437,547,350,565,468,348,...,441,66,1.474460,3.264876,9.741969,17.904160,17.746182,15.429173,23.486045,11.005793
6,1008,82,01,Ambutrix,739,156,127,165,168,89,...,127,4,0.000000,2.702703,12.162162,18.750000,18.074324,18.074324,21.452703,8.783784
7,1009,82,01,Andert-et-Condon,338,65,41,70,61,68,...,84,0,1.556420,4.669261,6.225681,9.338521,14.007782,21.789883,34.241245,7.782101
8,1010,82,01,Anglefort,1069,259,158,246,179,156,...,256,4,0.507614,4.568528,5.076142,13.197970,17.766497,18.274112,32.487310,8.121827
9,1011,82,01,Apremont,385,86,50,87,83,70,...,80,0,0.000000,5.263158,22.368421,9.210526,11.842105,15.789474,26.315789,9.210526


### Now we try to merge the 3 data frames: right_party_vote, immigration_data, salary_data

In [18]:
right_party_vote = pd.read_excel("right_party_vote.xlsx")
left_party_vote = pd.read_excel("left_party_vote.xlsx")
er_party_vote = pd.read_excel("er_party_vote.xlsx")

In [19]:
immigration_salary = pd.merge(immigration_data, salary_data, on = ['Code Insee','Libelle de la commune'], how = 'outer')

In [20]:
immigration_salary_population = pd.merge(population_data, immigration_salary, on = ['Code Insee','Libelle de la commune'], how = 'outer')

In [21]:
full_data_right = pd.merge(right_party_vote, immigration_salary_population, on = 'Code Insee', how = 'left')
full_data_left = pd.merge(left_party_vote, immigration_salary_population, on = 'Code Insee', how = 'left')
full_data_er = pd.merge(er_party_vote, immigration_salary_population, on = 'Code Insee', how = 'left')

In [24]:
full_data_right.head()

,Code Insee,Voix,% Voix/Ins,% Voix/Exp,% Abs/Ins,REG,DEP,Libelle de la commune,P12_POP,P12_POP0014,...,SNHMHO12,SNHM1812,SNHM2612,SNHM5012,SNHMF1812,SNHMF2612,SNHMF5012,SNHMH1812,SNHMH2612,SNHMH5012
0,1001,107,18.32,33.44,44.69,82,01,L'Abergement-Clemenciat,777,178,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1002,32,14.28,25.20,41.52,82,01,L'Abergement-de-Varey,235,44,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1004,1096,13.45,29.76,53.34,82,01,Amberieu-en-Bugey,14233,3038,...,10.945976,9.724445,13.042454,15.461354,9.049192,11.33007,12.313855,10.194692,14.088531,17.557177
3,1005,168,14.38,31.22,52.23,82,01,Amberieux-en-Dombes,1642,338,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1006,22,20.56,42.31,49.53,82,01,Ambleon,110,12,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Export the full data frame

In [22]:
writer = pd.ExcelWriter('full_data_right.xlsx')
full_data_right.to_excel(writer,'Sheet1')
writer.save()

KeyboardInterrupt: 

In [ ]:
writer = pd.ExcelWriter('full_data_left.xlsx')
full_data_left.to_excel(writer,'Sheet1')
writer.save()

In [ ]:
writer = pd.ExcelWriter('full_data_er.xlsx')
full_data_er.to_excel(writer,'Sheet1')
writer.save()